In [ ]:
import pandas as pd

data = pd.read_csv('df_mice.csv')
data = data.iloc[:,1:]
data.shape

In [ ]:
data['Sittime'] = data['BE8_1'] * 60 + data['BE8_2'] 
data.shape

In [ ]:
data.drop(['age','HE_ht','HE_wt', 'BE8_1', 'BE8_2'],axis=1,inplace=True)
data.shape

In [ ]:
#근력운동 일수 [BE5_1] - 전처리 필요X

In [ ]:
#1주일간 걷기 일수 [BE3_31]
data['BE3_31'].replace({7:6, 8:6}, inplace=True)
data['BE3_31'].value_counts()

In [ ]:
#앉아서 보내는 시간[Sittime]
data.loc[(0 <= data['Sittime'])  &  (data['Sittime'] < 180), 'Sittime'] = 1
data.loc[(180 <= data['Sittime'])  &  (data['Sittime'] < 360), 'Sittime'] = 2
data.loc[(360 <= data['Sittime'])  &  (data['Sittime'] < 540), 'Sittime'] = 3
data.loc[(540 <= data['Sittime'])  &  (data['Sittime'] < 720), 'Sittime'] = 4
data.loc[(720 <= data['Sittime'])  &  (data['Sittime'] < 900), 'Sittime'] = 5
data.loc[(900 <= data['Sittime']), 'Sittime'] = 6

data['Sittime'].value_counts()

In [ ]:
df = data.copy()
df.shape

In [ ]:
alpha = 0.02
low_limit = df.BMI.quantile(alpha)
high_limit = df.BMI.quantile(1-alpha)
BMI_trim = df[(df.BMI > low_limit) & (df.BMI < high_limit)]
BMI_trim['BMI'].describe()

In [ ]:
df['BMI'].sort_values().tail()

In [ ]:
'''df.loc[df['BMI'] < 18.5, 'BMI'] = 0
df.loc[df['BMI'] >= 18.5, 'BMI'] = 1
df['BMI'].value_counts()'''

# Train / Test

In [ ]:
# 독립변수와 종속변수 분류
X = df.drop('BMI',axis=1)
y = df[['BMI']]
print(X.shape, y.shape)

In [ ]:
# train vs test 나누기
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 0) 
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

In [ ]:
train = pd.concat([train_x, train_y], axis=1)
train.shape

In [ ]:
#train.to_csv('rf1.csv')

# RF

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, GridSearchCV
#rf = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=123456)
#rf.fit(X_train, y_train)

#최적 파라미터 값 찾기
params = [
        {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
        {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
    ]
#params =  {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8],
#        'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}
rf_clf = RandomForestRegressor(random_state = 0, n_jobs = -1)
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv = 3, scoring='neg_mean_squared_error', n_jobs = -1)
grid_cv.fit(train_x, train_y)

print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(bootstrap = True, max_features = 2, n_estimators = 30, random_state=0)
regr.fit(train_x, train_y)

In [ ]:
mse_list = []
for i in range(0, test_x.shape[0]):
    mse_list.append((abs(float(test_y.iloc[i,:]) - float(regr.predict([test_x.iloc[i,]])))) ** 2)
    
sum(mse_list) / test_x.shape[0]

In [ ]:
perfo_df = pd.DataFrame(columns=['y', 'pred_y'])
for i in range(0, test_x.shape[0]):
    perfo_df = perfo_df.append(pd.DataFrame([[float(train_y.iloc[i,:]), float(regr.predict([train_x.iloc[i,]]))]], columns=['y', 'pred_y']), ignore_index=True)
#df.set_index('idx', inplace=True)
perfo_df

In [ ]:
perfo_df.sort_values(by='pred_y').tail(30)

In [ ]:
import numpy as np

output_list = []
for i in range(0,1000):
    input1 = np.array(X.iloc[i,:])
    input1 = input1.reshape(1,-1)
    input1

    output_list.append(regr.predict(input1))

In [ ]:
print(output_list.index(max0))
print(output_list.index(max1))
print(output_list.index(max2))
print(output_list.index(max3))
print(output_list.index(max4))
print(output_list.index(max5))

In [ ]:
output_list.sort(reverse=True)
max0 = output_list[0]
max1 = output_list[1]
max2 = output_list[2]
max3 = output_list[3]
max4 = output_list[4]
max5 = output_list[5]
#x = output_list[1]

In [ ]:
import numpy as np
regr.predict(input1)

# SVM

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

param = {'kernel' : ('linear', 'poly', 'rbf', 'sigmoid'),
         'C' : [1,5,10],
         'degree' : [3,8],
         'coef0' : [0.01,10,0.5],
         'gamma' : ('auto','scale')},

model_svr = SVR()


grid_search = GridSearchCV(model_svr ,param_grid = param, cv=3, return_train_score = True)
grid_search.fit(train_x, train_y)

print('최적 하이퍼 파라미터: ', grid_search.best_params_)
#print('최고 예측 정확도: {:.4f}'.format(grid_search.best_score_))

In [ ]:
from sklearn.svm import SVR
svr = SVR(kernel='rbf', C=1, degree=3, coef0=0.01, gamma='auto')
svr.fit(train_x, train_y)

In [ ]:
mse_list = []
for i in range(0, test_x.shape[0]):
    mse_list.append((abs(float(test_y.iloc[i,:]) - float(svr.predict([test_x.iloc[i,]])))) ** 2)
    
sum(mse_list) / test_x.shape[0]

In [ ]:
perfo_df = pd.DataFrame(columns=['y', 'pred_y'])
for i in range(0, test_x.shape[0]):
    perfo_df = perfo_df.append(pd.DataFrame([[float(test_y.iloc[i,:]), float(svr.predict([test_x.iloc[i,]]))]], columns=['y', 'pred_y']), ignore_index=True)
#df.set_index('idx', inplace=True)
perfo_df

In [ ]:
perfo_df.sort_values(by='pred_y').tail(30)

# pickle

In [ ]:
import pickle

with open('./regr.pkl', 'wb') as f:
    pickle.dump(regr, f)

In [ ]:
with open('regr.pkl', 'rb') as f:
    data = pickle.load(f)